In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [3]:
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
#Scrape NASA featured Mars image

nasa_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(nasa_image_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

browser.click_link_by_id('full_image')
time.sleep(1)
browser.click_link_by_partial_text('more info')
time.sleep(1)
browser.click_link_by_partial_href('/spaceimages/images/largesize')

featured_image_url = browser.url
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17832_hires.jpg


In [6]:
#Scrape Mars weather
mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(mars_weather_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

time.sleep(1)
mars_weather = soup.find('p', class_='js-tweet-text').text

print(mars_weather)


# Tweet-class: js-stream-item stream-item stream-item


Three views of #ISS pass tonight as seen from The Hague, NL. 7 x 0.2 ms unfiltered mono in Celestron 8" at f/10.pic.twitter.com/5PvF1ECjjf


In [11]:
#Scrape facts table

mars_facts_url = 'https://space-facts.com/mars/#facts'
browser.visit(mars_facts_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

facts_table_scrape = soup.find('table', class_='tablepress-id-mars')

facts_df = pd.read_html(str(facts_table_scrape))[0]
facts_conversion_table = facts_df.to_html()

print(facts_conversion_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorde

In [8]:
#Scrape images - the large ones - I'm really not sure why we want giant TIFFs, but the client is always right! (Yeah but I grabbed the JPEGs too anyway because I am not sure the client is really 100% right in this case. They can have their TIFF URLs but to save myself the time of making updates later I'm going to also add some JPEGs to the dictionary. Just in case.)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

hemispheres_images_home_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_images_home_url)

hemisphere_links = soup.find_all('a', class_='product-item')

all_links = []
hemi_list = []
hemi_title_list = []
hemi_tiff_list = []
hemi_jepg_list = []

for link in set(hemisphere_links):
    current_link = link.get('href')
    all_links.append(current_link)
    
unique_links = list(set(all_links))

for i in range(len(unique_links)):
    hemi_dict = {}
    browser.visit((f'https://astrogeology.usgs.gov{unique_links[i]}'))
    print(f'https://astrogeology.usgs.gov{unique_links[i]}')
    time.sleep(2)
    hemi_title_raw = browser.find_by_css('h2.title').text
    hemi_title_final = hemi_title_raw.replace(' Enhanced', '')
    tiff_url = browser.find_link_by_text('Original')['href']
    jpeg_url = browser.find_link_by_text('Sample')['href']
    hemi_dict['title'] = hemi_title_final
    hemi_dict['tiff_url'] = tiff_url
    hemi_dict['img_url'] = jpeg_url
    print(hemi_dict)
    hemi_list.append(hemi_dict)
    
